# Тестовые данные

In [1]:
import pandas as pd
import numpy as np

In [2]:
!head -10 test.csv

<review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.
</review>

т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.
работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас , но на солнце 

In [3]:
!head -5 sample_submission.csv

Id,y
0,neg
1,neg
2,neg
3,neg


In [4]:
import codecs

In [5]:
with codecs.open("test.csv", "r", "utf8") as f:
    s = f.read()

In [6]:
from lxml import html

In [7]:
d = html.document_fromstring(s)

In [8]:
texts = list(d.xpath("//review/text()"))

In [9]:
len(texts)

100

In [10]:
print texts[0]

Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.



In [11]:
target = pd.read_csv("target.csv", header=None)

In [12]:
target.shape

(100, 1)

# Данные для обучения модели

Источник данных http://study.mokoron.com/

In [13]:
neg = pd.read_csv("negative.csv", sep=";", header=None)

In [14]:
neg.shape

(111923, 12)

In [15]:
neg.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0


In [16]:
print neg[3].values[0]

на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:


In [17]:
pos = pd.read_csv("positive.csv", sep=";", header=None)

In [18]:
pos.shape

(114911, 12)

In [19]:
pos.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2


In [20]:
from sklearn.linear_model import SGDClassifier

In [21]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [22]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [23]:
X = np.vstack([neg[3].values.reshape(-1, 1), pos[3].values.reshape(-1, 1)]).flatten()

In [24]:
y = np.vstack([np.zeros(neg.shape[0]).reshape(-1, 1), np.ones(pos.shape[0]).reshape(-1, 1)]).flatten()

In [25]:
from collections import Counter

In [26]:
Counter(y.flatten())

Counter({0.0: 111923, 1.0: 114911})

In [27]:
y.shape, X.shape

((226834,), (226834,))

In [29]:
from sklearn.model_selection import train_test_split

Данных очень много и их обработка занимает много времени. Выберем случайным образом относительно небольшое подмножество данных для быстрой проверки гипотез.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, random_state=0, test_size=0.1)

In [33]:
len(y_test)

22684

In [35]:
p = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(2, 2), analyzer="char_wb")),
    ("clf", SGDClassifier(loss="log", n_jobs=-1, random_state=0, max_iter=1e4, tol=1e-4, class_weight="balanced"))
])

Данная конфигурация была получена методом подбора. Оставлен наилучший вариант. Можно было бы использовать GridSearchCV, но в данном случае, подбирая параметры вручную, удобно корректировать перебираемые параметры в процессе перебора.

In [108]:
%%time
cross_val_score(p, X_test, y_test, cv=cv, n_jobs=-1).mean()

CPU times: user 503 ms, sys: 164 ms, total: 667 ms
Wall time: 9.4 s


0.99823660356940425

In [113]:
%%time
cross_val_score(p, X, y, cv=cv, n_jobs=-1).mean()

CPU times: user 3.78 s, sys: 510 ms, total: 4.29 s
Wall time: 1min 46s


0.9980426215525533

In [36]:
%%time
p.fit(X, y)

CPU times: user 17.6 s, sys: 240 ms, total: 17.9 s
Wall time: 17.9 s


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), norm=u'l2', preprocessor=None, smooth_idf=T...y='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=0.0001, verbose=0, warm_start=False))])

In [39]:
from sklearn.metrics import classification_report

In [40]:
print classification_report(target.values.flatten(), p.predict(texts))

             precision    recall  f1-score   support

          0       0.83      0.57      0.67        51
          1       0.66      0.88      0.75        49

avg / total       0.75      0.72      0.71       100



In [41]:
from sklearn.metrics import accuracy_score

In [42]:
accuracy_score(target.values.flatten(), p.predict(texts))

0.71999999999999997

Очевидно обучающая и тестовая выборка существенно отличаются. Возможно модель можно настроить под тестовые варьируя порог.

In [67]:
predicted = (p.predict_proba(texts)[:, 1] > 0.63).astype(int)

In [68]:
print classification_report(target.values.flatten(), predicted)

             precision    recall  f1-score   support

          0       0.69      0.94      0.79        51
          1       0.90      0.55      0.68        49

avg / total       0.79      0.75      0.74       100



In [69]:
accuracy_score(target.values.flatten(), predicted)

0.75

Мы улучшили accuracy score, но этого недостаточно. Бессмысленно пытаться улучшить результат на имеющейся обучающей выборке: построенная модель показывает очень высокий результат на кросс валидации, но не работает на тестовых данных. Нужна другая обучающая выборка.

# Новые данные для обучения

Обучающую выборку можно получить с сайта отзывов на модели мобильных телефонов, например https://torg.mail.ru/review/goods/mobilephones/

In [18]:
import requests

In [19]:
from lxml import etree

In [2]:
url = "https://torg.mail.ru/review/goods/mobilephones/"

In [3]:
page = requests.get(url).text

In [6]:
doc = html.document_fromstring(page)

In [26]:
scores = []
reviews = []
for review in doc.xpath('//div[@class="review-item__body"]'):
    review = etree.ElementTree(review)
    score = review.xpath('//span[@class="review-item__rating-counter"]/text()')
    score = int(score[0])
    scores.append(score)
    full = review.xpath('//p[@class="review-item__paragraph"]/a/@full-text')
    text = " ".join(full).replace("\n", " ")
    reviews.append(text)

In [20]:
from time import sleep

In [41]:
def scrape(page_num, score_file, review_file):
    url = "https://torg.mail.ru/review/goods/mobilephones/?page=%s" % page_num
    r = requests.get(url)
    r.raise_for_status()
    page = r.text
    doc = html.document_fromstring(page)
    for review in doc.xpath('//div[@class="review-item__body"]'):
        review = etree.ElementTree(review)
        score = review.xpath('//span[@class="review-item__rating-counter"]/text()')
        score_file.write("%s\n" % score[0])
        full = review.xpath('//p[@class="review-item__paragraph"]/a/@full-text')
        text = " ".join(full).replace("\r\n", " ").replace("<br />", " ")
        review_file.write(u"<review>%s</review>\n\n" % text)

In [ ]:
try:
    scores_file = codecs.open("scores.csv.raw", "a", "utf8")
    reviews_file = codecs.open("reviews.txt.raw", "a", "utf8")
    for page_num in xrange(1, 13395):
        scrape(page_num, scores_file, reviews_file)
        sleep(0.1)
finally:
    scores_file.close()
    reviews_file.close()

In [30]:
with codecs.open("reviews.txt.raw", "r", "utf8") as f:
    reviews = f.read()

In [11]:
import regex as re

Полученные ревью содержат html теги. Их нужно удалить. Так же некоторые ревью оказались пустыми. Не имеет смысла их учитывать.

In [31]:
snbr = re.sub("<br[^>]*>", " ", reviews, flags=re.IGNORECASE)

In [33]:
snbr = snbr.replace("\n", " ")

In [36]:
doc = html.document_fromstring(snbr)

In [37]:
reviews = doc.xpath('//review')

In [79]:
scores = pd.read_csv("scores.csv.raw", sep=";", header=None)

In [83]:
len(reviews), len(scores)

(17982, 17982)

In [38]:
reviews = [etree.ElementTree(r).xpath("//text()") or [""] for r in reviews]

In [39]:
reviews = np.array(reviews).flatten()

In [40]:
X_train = reviews[reviews != ""]

In [103]:
y_train = scores.values.flatten()[reviews != ""]

In [106]:
y_train = map(lambda v: float(v.replace(",", ".")), y_train)

In [110]:
y_train = np.array(y_train)

In [109]:
len(y_train), sum(np.array(y_train) != 3)

(7789, 7409)

Будем считать 3 звезды положительным ответом.

In [112]:
y_train = (y_train >= 3).astype(int)

Классы получились очень несбалансированными.

In [115]:
Counter(y_train)

Counter({0: 533, 1: 7256})

In [131]:
X_train = map(lambda v: v.replace("\n", " "), X_train)

In [41]:
with codecs.open("reviews.txt", "w", "utf8") as f:
    for r in X_train:
        f.write(r)
        f.write("\n")

In [133]:
with open("scores.csv", "w") as f:
    for s in y_train:
        f.write("%s\n" % s)

Так как обьемы данных достаточно большие может понадобиться сохранение данных в файл, перезапуск notebook-а и чтение данных из файла, чтоб избежать memory error.

# Тренировка на новых данных

In [21]:
X_train = pd.read_csv("reviews.txt", sep="@@@@@", header=None, engine="python").values.flatten()

In [59]:
y_train = pd.read_csv("scores.csv", header=None).values.flatten()

In [86]:
p = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 4))),
    ("clf", SGDClassifier(loss="log", n_jobs=-1, random_state=0, max_iter=1e4, tol=1e-4, class_weight="balanced"))
])

In [82]:
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [87]:
%%time
cross_val_score(p, X_train, y_train, cv=cv5, n_jobs=-1).mean()

CPU times: user 282 ms, sys: 169 ms, total: 452 ms
Wall time: 55.7 s


0.92206910641266493

In [97]:
p.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm=u'l2', preprocessor=None, smooth_idf=Tru...y='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=0.0001, verbose=0, warm_start=False))])

# Проверка на тестовых

In [98]:
print classification_report(target.values.flatten(), p.predict(texts))

             precision    recall  f1-score   support

          0       1.00      0.20      0.33        51
          1       0.54      1.00      0.71        49

avg / total       0.78      0.59      0.51       100



In [99]:
accuracy_score(target.values.flatten(), p.predict(texts))

0.58999999999999997

# 3 звезды как негативный отзыв

In [100]:
with codecs.open("reviews.txt.raw", "r", "utf8") as f:
    reviews = f.read()

In [101]:
snbr = re.sub("<br[^>]*>", " ", reviews, flags=re.IGNORECASE)

In [102]:
snbr = snbr.replace("\n", " ")

In [103]:
doc = html.document_fromstring(snbr)

In [104]:
reviews = doc.xpath('//review')

In [105]:
scores = pd.read_csv("scores.csv.raw", sep=";", header=None)

In [106]:
len(reviews), len(scores)

(17982, 17982)

In [107]:
reviews = [etree.ElementTree(r).xpath("//text()") or [""] for r in reviews]

In [108]:
reviews = np.array(reviews).flatten()

In [109]:
X_train = reviews[reviews != ""]

In [110]:
y_train = scores.values.flatten()[reviews != ""]

In [111]:
y_train = map(lambda v: float(v.replace(",", ".")), y_train)

In [112]:
y_train = np.array(y_train)

In [113]:
y_train = (y_train > 3).astype(int)

In [114]:
Counter(y_train)

Counter({0: 913, 1: 6876})

In [115]:
with open("scores3.csv", "w") as f:
    for s in y_train:
        f.write("%s\n" % s)

In [22]:
y_train = pd.read_csv("scores3.csv", header=None).values.flatten()

In [23]:
p = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 4))),
    ("clf", SGDClassifier(loss="log", n_jobs=-1, random_state=0, max_iter=1e4, tol=1e-4, class_weight="balanced"))
])

In [24]:
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [25]:
%%time
cross_val_score(p, X_train, y_train, cv=cv5, n_jobs=-1).mean()

CPU times: user 267 ms, sys: 116 ms, total: 383 ms
Wall time: 55.5 s


0.88560427178666445

In [26]:
p.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm=u'l2', preprocessor=None, smooth_idf=Tru...y='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=0.0001, verbose=0, warm_start=False))])

In [27]:
print classification_report(target.values.flatten(), p.predict(texts))

             precision    recall  f1-score   support

          0       1.00      0.43      0.60        51
          1       0.63      1.00      0.77        49

avg / total       0.82      0.71      0.69       100



In [30]:
accuracy_score(target.values.flatten(), p.predict(texts))

0.70999999999999996

# Позитивный отзыв с 4х звезд

In [31]:
with codecs.open("reviews.txt.raw", "r", "utf8") as f:
    reviews = f.read()

In [34]:
snbr = re.sub("<br[^>]*>", " ", reviews, flags=re.IGNORECASE)

In [35]:
snbr = snbr.replace("\n", " ")

In [36]:
doc = html.document_fromstring(snbr)

In [37]:
reviews = doc.xpath('//review')

In [38]:
scores = pd.read_csv("scores.csv.raw", sep=";", header=None)

In [39]:
len(reviews), len(scores)

(17982, 17982)

In [40]:
reviews = [etree.ElementTree(r).xpath("//text()") or [""] for r in reviews]

In [41]:
reviews = np.array(reviews).flatten()

In [42]:
y_train = scores.values.flatten()[reviews != ""]

In [43]:
y_train = map(lambda v: float(v.replace(",", ".")), y_train)

In [44]:
y_train = np.array(y_train)

In [45]:
y_train = (y_train >= 4).astype(int)

In [46]:
Counter(y_train)

Counter({0: 1438, 1: 6351})

In [47]:
with open("scores4.csv", "w") as f:
    for s in y_train:
        f.write("%s\n" % s)

In [48]:
y_train = pd.read_csv("scores4.csv", header=None).values.flatten()

In [49]:
p = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 4))),
    ("clf", SGDClassifier(loss="log", n_jobs=-1, random_state=0, max_iter=1e4, tol=1e-4, class_weight="balanced"))
])

In [50]:
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [51]:
%%time
cross_val_score(p, X_train, y_train, cv=cv5, n_jobs=-1).mean()

CPU times: user 275 ms, sys: 165 ms, total: 440 ms
Wall time: 59 s


0.86532234941524844

In [52]:
p.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm=u'l2', preprocessor=None, smooth_idf=Tru...y='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=0.0001, verbose=0, warm_start=False))])

# Варьируя порог

In [76]:
predicted = (p.predict_proba(texts)[:, 1] > 0.61).astype(int)

In [77]:
print classification_report(target.values.flatten(), predicted)

             precision    recall  f1-score   support

          0       0.98      0.86      0.92        51
          1       0.87      0.98      0.92        49

avg / total       0.93      0.92      0.92       100



In [78]:
accuracy_score(target.values.flatten(), predicted)

0.92000000000000004

In [79]:
answer = pd.DataFrame({"y": predicted})

In [82]:
answer = answer.assign(y=answer.y.apply(lambda v: "pos" if v else "neg"))

In [83]:
answer.to_csv("sentiment-w6-submission.csv", index_label="Id")

In [84]:
!head -5 sentiment-w6-submission.csv

Id,y
0,neg
1,pos
2,neg
3,neg


# Заключение

Было построено 2 модели, обученные на разных данных. Модель на основе отзывов на мобильные устройства показала более высокий результат на тестовых данных чем модель на основе twitter сообщений. Обе модели показали более хороший результат при увеличении порога классификации для позитивных отзывов.